In [ ]:
import torch
import tensorflow as f

import torchvision
import torchvision.transforms as transforms

import torch.nn as nn
import torch.nn.functional as F

In [ ]:
!pip install q torchinfo

from torchinfo import summary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
class LogisticRegression(nn.Module):
  def __init__(self, in_channels):
    super().__init__()

    self.log_reg = nn.Sequential(
        nn.Linear(in_channels, 1),
        nn.Sigmoid()
    )

  def forward(self, x):
    return self.log_reg(x)

In [ ]:
model = LogisticRegression(3)

criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01)

n_epoches = 10

x = torch.randn((1, 3)) ## (1,3) * (3,1) => (1,1)
y = torch.randn((1, 1 ))

for _ in range(n_epoches):
  optimizer.zero_grad()

  y_pred = model(x)

  loss = criterion(y_pred, y)

  loss.backward()
  optimizer.step() ## x = x - lr * gradient(x)


In [ ]:
class VGG(nn.Module):
  def __init__(self, num_class = 10, model_type = 'VGG19'):
    super().__init__()

    self.conv_arch = {"VGG11": [(1,64),(1,128),(2,256), (2,512), (2,512)],
                      "VGG19": [(2,64),(2,128),(4,256), (4,512), (4,512)],
                      'VGG16': [(2,64),(2,128),(3,256), (3,512), (3,512)]} # ASSIGNMENT

    self.num_class = num_class
    self.model_type = model_type

    self.layer = self.make_layers(self.model_type)


  def VGG_block(self, num_conv, in_channels, out_channels ):
    layers = []

    for _ in range(num_conv):
      layers.append( nn.Conv2d(in_channels, out_channels, kernel_size = 3, stride = 1, padding = 1))
      layers.append( nn.ReLU() )
      in_channels = out_channels

    layers.append(nn.MaxPool2d(kernel_size = 2, stride = 2))

    return nn.Sequential(*layers)


  def make_layers(self, type):
    conv_blocks = []

    in_channels = 3 # (batchsize , 3 , height = 224 ,width = 224) 112 56 28 14 7

    for (num_convs , out_channels) in self.conv_arch[type]:
      conv_blocks.append( self.VGG_block(num_convs, in_channels, out_channels))
      in_channels = out_channels


    layers = nn.Sequential(
        *conv_blocks,
        nn.Flatten(),
        nn.Linear(out_channels * 7 * 7, 4096),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(4096, 4096),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(4096, self.num_class)
    )

    return layers


  def forward(self, x):
    return self.layer(x)




In [ ]:
model = VGG(10, model_type = 'VGG16')
model

VGG(
  (layer): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU()
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU()
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (1): Sequential(
      (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU()
      (2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU()
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (2): Sequential(
      (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU()
      (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU()
      (4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (5): ReLU()
      (6): MaxPool2d(kernel_size=2, stride=2, paddin

In [ ]:
model = VGG(10, model_type = 'VGG16')

test_img = torch.randn((1,3, 224, 224 ))
print(test_img.shape)

torch.Size([1, 3, 224, 224])


In [ ]:
model(test_img).shape

torch.Size([1, 10])

In [ ]:
summary( model = VGG(10,model_type = 'VGG16'),
         input_size = (1,3,224,224),
        col_names = ['input_size', 'output_size', 'num_params', 'trainable'],
        row_settings = ['var_names'])

Layer (type (var_name))                  Input Shape               Output Shape              Param #                   Trainable
VGG (VGG)                                [1, 3, 224, 224]          [1, 10]                   --                        True
├─Sequential (layer)                     [1, 3, 224, 224]          [1, 10]                   --                        True
│    └─Sequential (0)                    [1, 3, 224, 224]          [1, 64, 112, 112]         --                        True
│    │    └─Conv2d (0)                   [1, 3, 224, 224]          [1, 64, 224, 224]         1,792                     True
│    │    └─ReLU (1)                     [1, 64, 224, 224]         [1, 64, 224, 224]         --                        --
│    │    └─Conv2d (2)                   [1, 64, 224, 224]         [1, 64, 224, 224]         36,928                    True
│    │    └─ReLU (3)                     [1, 64, 224, 224]         [1, 64, 224, 224]         --                        --
│    │ 